#### Benchmarks to motivate switching to PySpice's NGSPICE interface over LTSPICE command line interface

In [2]:
import numpy as np

import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

from os import system
import ltspice

In [12]:
def sim_ltspice(n):
    circuit = Circuit('test')

    circuit.V(1, 1, 0, f'{{V1}}')
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)


    xs = np.random.randn(n)
    ys = np.empty(n)

    for i, x in enumerate(xs):
        circuit.parameter('V1', x)
        simulator = circuit.simulator()
        analysis = simulator.operating_point()
        ys[i] = analysis['2'][0]

        netlist = str(circuit)

        with open(f'{circuit.title}.cir', 'wt') as f:
            f.write(netlist + '.op\n')

        system(f'/Applications/LTspice.app/Contents/MacOS/LTspice -b {circuit.title}.cir')
        
        l = ltspice.Ltspice(f'{circuit.title}.raw')
        l.parse()
        ys[i] = l.get_data('V(2)')[0]

    return ys


def sim_loop(n):
    circuit = Circuit('test')

    circuit.V(1, 1, 0, f'{{V1}}')
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)

    xs = np.random.randn(n)
    ys = np.empty(n)

    for i, x in enumerate(xs):
        circuit.parameter('V1', x)
        simulator = circuit.simulator()
        analysis = simulator.operating_point()
        ys[i] = analysis['2'][0]

    return ys

def sim_sweep(n):
    circuit = Circuit('test')

    circuit.V('dummy', 'dummy', 0, -1)
    
    circuit.R(1, 2, 1, 5)
    circuit.R(2, 2, 0, 1)

    xs = np.random.randn(n)
    arr = [str(val) for pair in zip(range(1, len(xs)+1), xs) for val in pair]
    ss = ', '.join(arr)
    s = f'pwl(V(dummy), {ss})' if n > 1 else xs[0]
    circuit.B(1, 1, 0, v=s)

    simulator = circuit.simulator()
    analysis = simulator.dc(Vdummy=slice(1, n, 1))

    return analysis['2']


##### Time benchmarks
Clearly, the NGSpice implementations are much faster (since they don't have to write to a file). It's much faster still to treat multiple evaluations on a single (fixed) circuit as parameter (voltage) sweeps over time and extract the results as such

In [126]:
%timeit sim_ltspice(100)

2.97 s ± 68.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit sim_loop(100)

624 µs ± 44.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
%timeit sim_sweep(100)

902 µs ± 37.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


We can see that all 3 methods are equivalent and yield correct results

In [125]:
sim_ltspice(100)

array([ 0.03753552,  0.07515843,  0.02449396, -0.2364496 , -0.13480079,
       -0.01133541,  0.338294  , -0.18403737, -0.29598153,  0.28983209,
        0.29039168,  0.29880744,  0.0798609 , -0.03220775, -0.14470004,
       -0.22870971, -0.12719533, -0.35808623, -0.09261359,  0.00710801,
       -0.03420829,  0.28246838, -0.14557856,  0.00908494,  0.03511909,
        0.07597739, -0.09303888,  0.0851147 ,  0.02474598,  0.14610714,
       -0.0972068 , -0.02091046,  0.27078012,  0.16743305,  0.25520605,
        0.08348022, -0.02364058, -0.13090456,  0.06406467,  0.0687949 ,
        0.16362356,  0.10467349,  0.04359397, -0.07487158,  0.22197866,
        0.06946041, -0.07002871,  0.0234651 ,  0.24606387,  0.06372093,
       -0.23425278,  0.06892099, -0.31014207, -0.05421357,  0.06740066,
        0.13865423, -0.19310217, -0.12016212, -0.14590812,  0.19561708,
        0.31243864, -0.00499533,  0.03768062,  0.00767007, -0.1543553 ,
        0.17335002,  0.34307778, -0.37501946,  0.15394862, -0.01

In [24]:
sim_loop(100)

array([ 0.01123329,  0.06062443, -0.09089946, -0.15238242,  0.08888068,
       -0.41207474, -0.14006066,  0.09362176, -0.03268056,  0.11438461,
        0.00701414, -0.04649972, -0.16279291,  0.09678377, -0.15019855,
        0.19665815, -0.08758584, -0.09212711,  0.01676941, -0.13284763,
       -0.10181434, -0.25951984, -0.01772411,  0.18525823,  0.08985063,
       -0.17553955,  0.1661906 , -0.02867808, -0.01711069, -0.07482609,
       -0.13034819,  0.25476729, -0.09541836, -0.1898226 , -0.09891867,
        0.01942286,  0.03253694, -0.08163798,  0.12893064, -0.13297498,
        0.05205943, -0.23653128, -0.19320342, -0.13068765, -0.01068468,
        0.29793971,  0.46016888,  0.10050677,  0.12837404,  0.09822345,
        0.30206613,  0.07689809, -0.05319078, -0.04039697,  0.17711811,
       -0.28438093, -0.02225215,  0.12328244, -0.06927709,  0.11035077,
        0.27362333, -0.0372224 , -0.23744233,  0.00239705,  0.02321959,
       -0.00694601, -0.1337243 ,  0.07607184,  0.12677499,  0.04

In [23]:
sim_sweep(100)

WaveForm  [-0.38747401 -0.28427913 -0.3047998   0.02029898 -0.09624381 -0.02996156
 -0.0508552  -0.15897451  0.12519814  0.03000011 -0.21740534 -0.36832264
 -0.22292482 -0.04414901 -0.24765621 -0.26914994  0.12883063 -0.05363416
 -0.34663943 -0.48129173 -0.1483392  -0.40408859  0.4407186  -0.13097377
  0.06304524  0.06613157 -0.15916389 -0.20969341 -0.07633052  0.22231879
 -0.18108803  0.22742817 -0.0552443  -0.0172125  -0.05115474  0.17951528
 -0.0340121  -0.50269107  0.03650035 -0.03261185 -0.16098776  0.00857163
  0.47276317 -0.35840804  0.09073933 -0.18626195  0.03961813  0.13399191
 -0.06123191 -0.12441521 -0.14214479 -0.15009528  0.1379168  -0.12201175
  0.4299848   0.26913507  0.39548844  0.11474079  0.00061503 -0.26342002
 -0.03734797  0.13927093  0.38633542  0.2320957   0.09794226 -0.54838657
  0.05017008 -0.00801883 -0.45191579 -0.14566971 -0.07846605 -0.04246098
  0.09830568 -0.24703759  0.18770736 -0.21835167  0.11303345  0.01723933
 -0.00913985 -0.04298726  0.13563887  0.1